<a href="https://colab.research.google.com/github/vietddude/bookmgmt/blob/master/CSC14118_Introduction_to_Spark_20_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Java and Spark on Hadoop

In [ ]:
# install java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# install spark (change the version number if needed)
!wget -q https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# unzip the spark file to the current folder
!tar xf spark-3.3.2-bin-hadoop3.tgz

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,058 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,354 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,255 

In [ ]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

In [ ]:
!pip install findspark
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Create a SparkSession in Python

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local")\
          .appName("Introduction to Spark")\
          .config("spark.some.config.option", "some-value")\
          .getOrCreate()

In [ ]:
# Import necessary libraries
from pyspark.sql.functions import col, column, expr
from pyspark.sql import functions as f

# Exercise 1: Data query with Spark DataFrame

---



##0. Load the data file: movies.json

In [ ]:
!git clone https://github.com/nnthaofit/CSC14118.git

fatal: destination path 'CSC14118' already exists and is not an empty directory.


In [ ]:
df = spark.read.json("CSC14118/movies.json")

In [ ]:
df.show(10, truncate=False)

+-------------+--------------------+-------------------------------------------+----+
|cast         |genres              |title                                      |year|
+-------------+--------------------+-------------------------------------------+----+
|[]           |[]                  |After Dark in Central Park                 |1900|
|[]           |[]                  |Boarding School Girls' Pajama Parade       |1900|
|[]           |[]                  |Buffalo Bill's Wild West Parad             |1900|
|[]           |[]                  |Caught                                     |1900|
|[]           |[]                  |Clowns Spinning Hats                       |1900|
|[]           |[Short, Documentary]|Capture of Boer Battery by British         |1900|
|[]           |[]                  |The Enchanted Drawing                      |1900|
|[Paul Boyton]|[]                  |Feeding Sea Lions                          |1900|
|[]           |[Comedy]            |How to Make a Fat 

## 1a. Show the schema of DataFrame that stores the movies dataset.

In [ ]:
df.schema

StructType([StructField('cast', ArrayType(StringType(), True), True), StructField('genres', ArrayType(StringType(), True), True), StructField('title', StringType(), True), StructField('year', LongType(), True)])

In [ ]:
df.printSchema()

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



## 1b. Show the number of distinct films in the dataset

In [ ]:
df.show(10)

+-------------+--------------------+--------------------+----+
|         cast|              genres|               title|year|
+-------------+--------------------+--------------------+----+
|           []|                  []|After Dark in Cen...|1900|
|           []|                  []|Boarding School G...|1900|
|           []|                  []|Buffalo Bill's Wi...|1900|
|           []|                  []|              Caught|1900|
|           []|                  []|Clowns Spinning Hats|1900|
|           []|[Short, Documentary]|Capture of Boer B...|1900|
|           []|                  []|The Enchanted Dra...|1900|
|[Paul Boyton]|                  []|   Feeding Sea Lions|1900|
|           []|            [Comedy]|How to Make a Fat...|1900|
|           []|                  []|     New Life Rescue|1900|
+-------------+--------------------+--------------------+----+
only showing top 10 rows



In [ ]:
df.distinct().count()

28789

## 2. Count the number of movies released during the years 2012 and 2015 (included)

In [ ]:
# Import the functions module
from pyspark.sql import functions as F

# Filter the movies by year
df_filtered = df.filter((F.col("year") >= 2012) & (F.col("year") <= 2015))

# Count the number of movies
movie_count = df_filtered.count()

# Print the result
print(f"There are {movie_count} movies released during the years 2012 and 2015.")

There are 1015 movies released during the years 2012 and 2015.


In [ ]:
df.filter(df.year>=2012).filter(df.year<=2015).count()

1015

## 3. Show the year in which the number of movies released is highest. One highest year is enough

In [ ]:
df1 = df.groupBy(df.year).agg({"title":"count"}).withColumnRenamed('count(title)','count')
df1.sort(df1["count"].desc()).show(1)

+----+-----+
|year|count|
+----+-----+
|1919|  634|
+----+-----+
only showing top 1 row



In [ ]:
df1=df.groupBy("year").count()

In [ ]:
df1.show()

+----+-----+
|year|count|
+----+-----+
|1950|  443|
|1919|  634|
|1936|  504|
|1951|  429|
|1958|  281|
|1921|  143|
|1983|  140|
|1905|   35|
|1972|  140|
|1979|  139|
|2007|  304|
|1988|  292|
|2014|  214|
|1908|   18|
|1986|  192|
|1949|  351|
|1930|  361|
|1969|  137|
|1964|  151|
|1967|  127|
+----+-----+
only showing top 20 rows



In [ ]:
df1.sort(col("count").desc()).show(1)

+----+-----+
|year|count|
+----+-----+
|1919|  634|
+----+-----+
only showing top 1 row



In [ ]:
f.max(df1['count'])

Column<'max(count)'>

## 4. Show the list of movies such that for each film, the number of actors/actresses is at least five, and the number of genres it belongs to is at most two genres.

In [ ]:
df.filter(f.size(df.cast)>=5).filter(f.size(df.genres)<=2).show()

+--------------------+----------------+--------------------+----+
|                cast|          genres|               title|year|
+--------------------+----------------+--------------------+----+
|[Earle Foxe, Alie...|         [Drama]|  A Desperate Chance|1913|
|[Charlotte Burton...|         [Drama]|    The Archeologist|1914|
|[Charlotte Burton...|         [Drama]|At the Potter's W...|1914|
|[Herbert Tracey, ...|        [Comedy]|    Back to the Farm|1914|
|[Charlotte Burton...|              []|    The Beggar Child|1914|
|[William Garwood,...|              []|       Billy's Rival|1914|
|[B. Reeves Eason,...|         [Drama]| Break, Break, Break|1914|
|[Charlotte Burton...|              []|       The Butterfly|1914|
|[Charlotte Burton...|       [Western]|Calamity Anne's L...|1914|
|[Charlie Chaplin,...|        [Comedy]|    The Star Boarder|1914|
|[Sydney Ayres, Ja...|              []|A Story of Little...|1914|
|[Sydney Ayres, Pe...|              []|The Story of the ...|1914|
|[Charlott

In [ ]:
df.select("cast").show(truncate=False)

+----------------+
|cast            |
+----------------+
|[]              |
|[]              |
|[]              |
|[]              |
|[]              |
|[]              |
|[]              |
|[Paul Boyton]   |
|[]              |
|[]              |
|[]              |
|[]              |
|[]              |
|[]              |
|[]              |
|[Ching Ling Foo]|
|[]              |
|[]              |
|[]              |
|[]              |
+----------------+
only showing top 20 rows



In [ ]:
df.where((f.size(df.cast)>=5)&(f.size(df.genres)<=2)).show()

+--------------------+----------------+--------------------+----+
|                cast|          genres|               title|year|
+--------------------+----------------+--------------------+----+
|[Earle Foxe, Alie...|         [Drama]|  A Desperate Chance|1913|
|[Charlotte Burton...|         [Drama]|    The Archeologist|1914|
|[Charlotte Burton...|         [Drama]|At the Potter's W...|1914|
|[Herbert Tracey, ...|        [Comedy]|    Back to the Farm|1914|
|[Charlotte Burton...|              []|    The Beggar Child|1914|
|[William Garwood,...|              []|       Billy's Rival|1914|
|[B. Reeves Eason,...|         [Drama]| Break, Break, Break|1914|
|[Charlotte Burton...|              []|       The Butterfly|1914|
|[Charlotte Burton...|       [Western]|Calamity Anne's L...|1914|
|[Charlie Chaplin,...|        [Comedy]|    The Star Boarder|1914|
|[Sydney Ayres, Ja...|              []|A Story of Little...|1914|
|[Sydney Ayres, Pe...|              []|The Story of the ...|1914|
|[Charlott

## 5. Show the **movies** whose names are longest

In [ ]:
df.sort(f.length(df.title).desc()).show(1,truncate=False)

+----+------+--------------------------------------------------------------------------------------------------------------+----+
|cast|genres|title                                                                                                         |year|
+----+------+--------------------------------------------------------------------------------------------------------------+----+
|[]  |[]    |Cornell-Columbia-University of Pennsylvania Boat Race at Ithaca, N.Y., Showing Lehigh Valley Observation Train|1901|
+----+------+--------------------------------------------------------------------------------------------------------------+----+
only showing top 1 row



In [ ]:
df.orderBy(f.length(col('title')).desc()).limit(1).show(truncate=False)

+----+------+--------------------------------------------------------------------------------------------------------------+----+
|cast|genres|title                                                                                                         |year|
+----+------+--------------------------------------------------------------------------------------------------------------+----+
|[]  |[]    |Cornell-Columbia-University of Pennsylvania Boat Race at Ithaca, N.Y., Showing Lehigh Valley Observation Train|1901|
+----+------+--------------------------------------------------------------------------------------------------------------+----+



## 6. Show the movies whose name contains the word “fighting” (case-insensitive).

In [ ]:
df.filter(f.lower(df.title).contains('fighting')).show(truncate=False)

+------------------------------------------+---------------+-----------------------+----+
|cast                                      |genres         |title                  |year|
+------------------------------------------+---------------+-----------------------+----+
|[Bessie Love, Anne Schaefer]              |[Comedy, Drama]|A Fighting Colleen     |1919|
|[Blanche Sweet, Russell Simpson]          |[Western]      |Fighting Cressy        |1919|
|[Harry T. Morey, Betty Blythe]            |[Drama]        |Fighting Destiny       |1919|
|[Tom Mix, Teddy Sampson]                  |[Western]      |Fighting for Gold      |1919|
|[Jack Perrin, Hoot Gibson, Josephine Hill]|[Western]      |The Fighting Heart     |1919|
|[Art Acord, Mildred Moore]                |[Western]      |The Fighting Line      |1919|
|[William Duncan, Edith Johnson]           |[Action]       |The Fighting Guide     |1922|
|[Tom Mix, Patsy Ruth Miller]              |[Western]      |The Fighting Streak    |1922|
|[Richard 

In [ ]:
df.filter(f.lower(col('title')).contains('fighting')).show()

+--------------------+---------------+--------------------+----+
|                cast|         genres|               title|year|
+--------------------+---------------+--------------------+----+
|[Bessie Love, Ann...|[Comedy, Drama]|  A Fighting Colleen|1919|
|[Blanche Sweet, R...|      [Western]|     Fighting Cressy|1919|
|[Harry T. Morey, ...|        [Drama]|    Fighting Destiny|1919|
|[Tom Mix, Teddy S...|      [Western]|   Fighting for Gold|1919|
|[Jack Perrin, Hoo...|      [Western]|  The Fighting Heart|1919|
|[Art Acord, Mildr...|      [Western]|   The Fighting Line|1919|
|[William Duncan, ...|       [Action]|  The Fighting Guide|1922|
|[Tom Mix, Patsy R...|      [Western]| The Fighting Streak|1922|
|[Richard Barthelm...|   [Historical]|  The Fighting Blade|1923|
|[Ernest Torrence,...|       [Comedy]| The Fighting Coward|1924|
|[Jack Hoxie, Hele...|      [Western]|       Fighting Fury|1924|
|[Pat O'Malley, Ma...|        [Drama]|The Fighting Adve...|1924|
|[Fred Thomson, Ha...|   

## 7. Show the list of distinct genres appearing in the dataset


In [ ]:
df.select(f.explode(df.genres)).distinct().show()

+-------------+
|          col|
+-------------+
|        Crime|
|      Romance|
|     Thriller|
|      Slasher|
|Found Footage|
|    Adventure|
|         Teen|
| Martial Arts|
|       Sports|
|        Drama|
|          War|
|  Documentary|
|       Family|
|      Fantasy|
|       Silent|
|     Disaster|
|        Legal|
|      Mystery|
| Supernatural|
|     Suspense|
+-------------+
only showing top 20 rows



In [ ]:
df.select(f.explode(col('genres'))).distinct().show()

+-------------+
|          col|
+-------------+
|        Crime|
|      Romance|
|     Thriller|
|      Slasher|
|Found Footage|
|    Adventure|
|         Teen|
| Martial Arts|
|       Sports|
|        Drama|
|          War|
|  Documentary|
|       Family|
|      Fantasy|
|       Silent|
|     Disaster|
|        Legal|
|      Mystery|
| Supernatural|
|     Suspense|
+-------------+
only showing top 20 rows



## 8. List all movies in which the actor Harrison Ford has participated.

In [ ]:
df.filter(f.array_contains(df.cast,"Harrison Ford")).show()

+--------------------+-----------------+--------------------+----+
|                cast|           genres|               title|year|
+--------------------+-----------------+--------------------+----+
|[Constance Talmad...|[Romance, Comedy]|Experimental Marr...|1919|
|[Constance Talmad...|         [Comedy]| Happiness a la Mode|1919|
|[Constance Talmad...|         [Comedy]|Romance and Arabella|1919|
|[Vivian Martin, H...|         [Comedy]|      The Third Kiss|1919|
|[Harrison Ford, C...|         [Comedy]|The Veiled Adventure|1919|
|[Constance Talmad...|         [Comedy]|          Who Cares?|1919|
|[Vivian Martin, H...|          [Drama]|You Never Saw Suc...|1919|
|[Norma Talmadge, ...|          [Drama]| The Wonderful Thing|1921|
|[Alma Rubens, Har...|        [Mystery]|      Find the Woman|1922|
|[Constance Talmad...|          [Drama]| The Primitive Lover|1922|
|[Norma Talmadge, ...| [Romance, Drama]|     Smilin' Through|1922|
|[Helen Jerome Edd...|          [Drama]|     When Love Comes|1

In [ ]:
df.filter(f.array_contains(col('cast'), 'Harrison Ford')).show()

+--------------------+-----------------+--------------------+----+
|                cast|           genres|               title|year|
+--------------------+-----------------+--------------------+----+
|[Constance Talmad...|[Romance, Comedy]|Experimental Marr...|1919|
|[Constance Talmad...|         [Comedy]| Happiness a la Mode|1919|
|[Constance Talmad...|         [Comedy]|Romance and Arabella|1919|
|[Vivian Martin, H...|         [Comedy]|      The Third Kiss|1919|
|[Harrison Ford, C...|         [Comedy]|The Veiled Adventure|1919|
|[Constance Talmad...|         [Comedy]|          Who Cares?|1919|
|[Vivian Martin, H...|          [Drama]|You Never Saw Suc...|1919|
|[Norma Talmadge, ...|          [Drama]| The Wonderful Thing|1921|
|[Alma Rubens, Har...|        [Mystery]|      Find the Woman|1922|
|[Constance Talmad...|          [Drama]| The Primitive Lover|1922|
|[Norma Talmadge, ...| [Romance, Drama]|     Smilin' Through|1922|
|[Helen Jerome Edd...|          [Drama]|     When Love Comes|1

## 9. List all movies in which the actors/actresses whose names include the word “Lewis“ (case-insensitive) have participated.

In [ ]:
df.withColumn('cast_str', f.array_join(col('cast'), ' ')) \
  .filter(f.lower(col('cast_str')).contains('lewis')) \
  .drop('cast_str') \
  .show()

+--------------------+-----------+--------------------+----+
|                cast|     genres|               title|year|
+--------------------+-----------+--------------------+----+
|[Charlotte Burton...|         []|       The Butterfly|1914|
|[Pearl White, She...|    [Drama]|The Exploits of E...|1914|
|[Charlotte Burton...|   [Comedy]| Mein Lieber Katrina|1914|
|[Norma Talmadge, ...|    [Drama]|      Going Straight|1916|
|[Dorothy Gish, Ra...|    [Drama]|Gretchen the Gree...|1916|
|[Ben Lewis, Bessi...|  [Western]|     A Sister of Six|1916|
|[Gail Kane, Lewis...|    [Drama]| The Bride's Silence|1917|
|    [Mitchell Lewis]|    [Drama]|Nine-Tenths of th...|1918|
|[Mitchell Lewis, ...|    [Drama]|The Faith of the ...|1919|
|[Mary Pickford, R...|   [Comedy]|         The Hoodlum|1919|
|[Mitchell Lewis, ...|    [Drama]|Jacques of the Si...|1919|
|[Mitchell Lewis, ...|    [Drama]|The Last of His P...|1919|
|[Lewis Stone, Jan...|    [Drama]|        Man's Desire|1919|
|[Mary Miles Minte...|  

## 10. Show top five actors/actresses that have participated in most movies.

In [ ]:
df1 = df.select(f.explode('cast').alias('cast'), 'title', 'year').groupby('cast').count()
df1.sort(df1['count'].desc()).show(5)

+----------------+-----+
|            cast|count|
+----------------+-----+
|    Harold Lloyd|  190|
|     Hoot Gibson|  142|
|      John Wayne|  136|
|Charles Starrett|  116|
|    Bebe Daniels|  103|
+----------------+-----+
only showing top 5 rows



# Exercise 2: RDD-based mainpulation


*   The data is already in one ore more RDDs.
*   You must not convert RDD to DF or use pure Python code.





## 1. Consider a string s that includes only alphabetical letters and spaces. Check whether s is a palindrome (case-insensitive).

In [ ]:
a = [('Aary', 1), ('had', 2), ('a', 3), ('little', 4), ('lamb', 5)]
a.extend([('whose', 6), ('fleece', 7), ('was', 8), ('white', 9)])
a

[('Aary', 1),
 ('had', 2),
 ('a', 3),
 ('little', 4),
 ('lamb', 5),
 ('whose', 6),
 ('fleece', 7),
 ('was', 8),
 ('white', 9)]

In [ ]:
rdd = spark.sparkContext.parallelize(a)
rdd.collect()

[('Aary', 1),
 ('had', 2),
 ('a', 3),
 ('little', 4),
 ('lamb', 5),
 ('whose', 6),
 ('fleece', 7),
 ('was', 8),
 ('white', 9)]

In [ ]:
rdd.sortByKey(True,3,keyfunc=lambda k: k.lower()).collect()

[('a', 3),
 ('Aary', 1),
 ('fleece', 7),
 ('had', 2),
 ('lamb', 5),
 ('little', 4),
 ('was', 8),
 ('white', 9),
 ('whose', 6)]

In [ ]:
rdd1 = spark.sparkContext.parallelize([12,34,12,44235,346,45,23,324,2354])
rdd1.stats()

(count: 9, mean: 5265.0, stdev: 13796.228172141027, max: 44235.0, min: 12.0)

In [ ]:
def isPalin(s):
    # remove non-alphabetical characters and lower
    string = ''.join(char for char in s if char.isalnum()).lower()
    words = spark.sparkContext.parallelize(string)
    numRange = spark.sparkContext.parallelize(range(len(string)))
    return numRange.zip(words).sortBy(lambda pair:pair[0] * -1).values().collect() == words.collect()

In [ ]:
print(isPalin('Yangchen'))
print(isPalin('Kuruk'))
print(isPalin('Kyoshi'))
print(isPalin('Roku'))
print(isPalin('Aang'))
print(isPalin('Korra'))

False
True
False
False
False
False


## 2. Consider a string s that includes only alphabetical letters and spaces. Check whether s is a pangram (case-insensitive).

In [ ]:
def isPangram(s):
    # remove non-alphabetical characters and lower
    string = ''.join(char for char in s if char.isalnum()).lower()
    words = spark.sparkContext.parallelize(string)
    return len(set(words.collect())) == 26 # we're assuming this is English string.

In [ ]:
isPangram('The quick brown fox jumps over the lazy dog')

True

In [ ]:
isPangram('The man who passes the sentence should swing the sword')

False

# Exercise 3: Frequent patterns and association rules mining

## 0. Load the data file: foodmart.csv


*  A record is a tuple of binary values {0, 1}, each of which denotes the presence of an item (1: bought, 0: not bought).



In [ ]:
!git clone https://github.com/nnthaofit/CSC14118.git

Cloning into 'CSC14118'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), 816.70 KiB | 2.12 MiB/s, done.


In [ ]:
df = spark.read.csv("CSC14118/foodmart.csv", header = True, inferSchema = True) 

In [ ]:
df.show()

+------------+---------+-------+--------------+------+---------+----+-------+-------+------------+-----------------+------+------+-----+---------+---------------+-----+--------+------+-------------+------------------+-----------+-------+-----------+--------------+--------+----------+-----------+-----------+----+------+-----------+----------+----+-----------------+---------------+------------+-------------+----------+--------------+-----------------+---+---------+----------+--------------+--------+---------+---------+---+-----+-----+----------+----+----+---------+-------+------------+----+-------+-----------+--------+------------+-----------+-----+-------------+----------------+-----+----------------+-------+---------+------------+-------------+-------------+---------+--------+----+--------+------+------------+-------+---------+------+------------+----+----+----------+------+-------+----------------+-----+----------+----+--------------+-----+------------+----+---------+-------+----+----

## 1. Convert the given data to the format required by Spark MLlib FPGrowth.

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import Row

row = Row("items")


# create Spark session
#spark = SparkSession.builder.appName("FPGrowthExample").getOrCreate()

# read CSV file
df = spark.read.csv("CSC14118/foodmart.csv", header=True, inferSchema=True)

# convert sets to lists
df = df.rdd.map(lambda x: tuple(list(x))).toDF(df.columns)

# transform dataframe to transactions
transactions = df.rdd.map(row).toDF()

# # use FPGrowth algorithm
fp = FPGrowth(minSupport=0.5, numPartitions=10, itemsCol="items")
model = fp.fit(transactions)

# # print frequent itemsets
model.freqItemsets.show()

IllegalArgumentException: ignored

In [ ]:
from pyspark.sql import Row
row = Row("items")
transactions = df.rdd.map(row).toDF()
transactions.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|items                                                                                                                                                                                                                                                                                                       |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Column<'size(items)'>

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id 

df_index = df.select("*").withColumn("id", monotonically_increasing_id())
df_index.show()

+------------+---------+-------+--------------+------+---------+----+-------+-------+------------+-----------------+------+------+-----+---------+---------------+-----+--------+------+-------------+------------------+-----------+-------+-----------+--------------+--------+----------+-----------+-----------+----+------+-----------+----------+----+-----------------+---------------+------------+-------------+----------+--------------+-----------------+---+---------+----------+--------------+--------+---------+---------+---+-----+-----+----------+----+----+---------+-------+------------+----+-------+-----------+--------+------------+-----------+-----+-------------+----------------+-----+----------------+-------+---------+------------+-------------+-------------+---------+--------+----+--------+------+------------+-------+---------+------+------------+----+----+----------+------+-------+----------------+-----+----------+----+--------------+-----+------------+----+---------+-------+----+----

In [ ]:
# !
df.show()

In [ ]:
data.show()

## 2.	Apply Spark MLlib FPGrowth to the formatted data. Mine the set of frequent patterns with the minimum support of 0.1. Mine the set of association rules with the minimum confidence of 0.9.

In [ ]:
# Assuming your data is already in the required format, with each transaction on a separate row and items in a column named 'items'
transactions = df.select("items")
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1)
model = fpGrowth.fit(transactions)
frequentPatterns = model.freqItemsets
associationRules = model.associationRules
associationRules = associationRules.filter(associationRules["confidence"] >= 0.9)
frequentPatterns.show()
associationRules.show()


AnalysisException: ignored

# Exercise 4: Classification

##0. Load the data file: mushroom.csv
*   The data represents a collection of mushroom species. 
*   There are 8124 examples, each of which has 22 attributes and it is categorized into either “edible” (e) or “poisonous” (p) 


In [ ]:
!git clone https://github.com/nnthaofit/CSC14118.git

fatal: destination path 'CSC14118' already exists and is not an empty directory.


In [ ]:
df = spark.read.csv("CSC14118/mushrooms.csv", header = True, inferSchema = True) 

In [ ]:
df.select('cap-surface').distinct().collect()

[Row(cap-surface='g'),
 Row(cap-surface='f'),
 Row(cap-surface='y'),
 Row(cap-surface='s')]

## 1.	Prepare the train and test sets following the ratio 8:2

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as f
from pyspark.ml.linalg import Vectors

In [ ]:
df_index = df.withColumn("id",f.monotonically_increasing_id())
df_index.show()

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+---+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat| id|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+---+
|    p|        x|          s|        n|      t|   p|              f|           c| 

In [ ]:
split = df_index.randomSplit([8.0,2.0],24)

In [ ]:
train = split[0].drop("id")
test = split[1].drop("id")

In [ ]:
feature = train.drop("class")
feature = feature.rdd.map(lambda x: (list(x))).toDF(feature.columns)

In [ ]:
from pyspark.sql import Row

row = Row("items")

a =feature.rdd.map(row).toDF()
a = a.rdd.map(list).toDF(["features"])

In [ ]:
a.features.apply(lambda row: [row])

TypeError: ignored

## 2. Fit a decision tree model on the training set, using Spark MLlib DecisionTreeClassifier with default parameters

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
dt = DecisionTreeRegressor()
model =dt.fit(train)

IllegalArgumentException: ignored

## 3. Fit a random forest model on the training set, using Spark MLlib RandomForestClassification with default parameters

## 4. Evaluate the two models on the same test set using the following metrics: areaUnderROC and areaUnderPR

##5. Chain the above steps into a single pipeline

# Exercise 5: Clustering

##0. Load the data file: iris.csv

*   The data represents a collection of iris plant species.
*   There are 150 examples, each of which has 4 attributes and it is categorized into one of the three classes.  

In [ ]:
!git clone https://github.com/nnthaofit/CSC14118.git

In [ ]:
df = spark.read.csv("CSC14118/iris.csv", header = True, inferSchema = True) 

## 1.	Cluster the data by using Spark MLlib KMeans with k = 2, 3, and 5, using Euclidean distance and cosine distance

## 2. Evaluate each of the above clustering results using silhoutte score. 

1.   List item
2.   List item

Which configuration yeilds the best clustering?

##3. Chain the above steps into a single pipeline

## 4. For each clustering result obtained above, count the number of examples that belong to each of the three species. 

# Exercise 6: Network manipulation with Spark GraphFrames

##0. Load the data files: users.txt and followers.txt

In [ ]:
!git clone https://github.com/nnthaofit/CSC14118.git

##1.	Construct a graph from the given data to demonstrate a tiny social network


##2.	Apply Graphs graphPageRank to the network to obtain a ranking list of users in terms of followers

##3. Find connected components on the graph, using Graphs connectedComponents or stronglyConnectedComponents